In [173]:
%matplotlib inline

In [174]:
path = "../../homework/data/kaggle_dogs_vs_cats/"

In [175]:
from __future__ import division,print_function

import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
from keras.preprocessing import image as keras_image

In [176]:
import utils; reload(utils)
from utils import plots

In [177]:
import vgg16; reload(vgg16)
from vgg16 import Vgg16

In [178]:
batch_size=64

In [179]:
# Prepare model and create batch
vgg = Vgg16()
batches = vgg.get_batches(path+'train', batch_size=batch_size)
val_batches = vgg.get_batches(path+'valid', batch_size=batch_size)

Found 16 images belonging to 2 classes.
Found 8 images belonging to 2 classes.


In [180]:
# Finetue the model on our data
vgg.finetune(batches)
vgg.fit(batches, val_batches, nb_epoch=1)

Epoch 1/1
16/16 [==============================] - 6s - loss: 2.1364 - acc: 0.5625 - val_loss: 0.4273 - val_acc: 0.8750


In [181]:
# test_batches = vgg.get_batches(path + 'mytest', shuffle=False, batch_size=2, class_mode=None)

In [182]:
# img_gen = keras_image.ImageDataGenerator()
# test_batches = img_gen.flow_from_directory(path + 'mytest', target_size=(224, 224), shuffle=False, batch_size=4, class_mode=None)

In [183]:
# imgs = next(test_batches)
# plots(imgs)

In [184]:
# type(imgs)
# imgs.shape

In [185]:
# vgg.predict(imgs, True)

In [186]:
# predict class for an image file, return 0 on dog and 1 on cat
def predict_file(path):
    img_obj = keras_image.load_img(path=path, target_size=(224, 224))
    img = keras_image.img_to_array(img_obj)
    res = vgg.predict(np.array([img]), True)
    s = res[2][0]
    if s == 'cats':
        return 1
    else:
        return 0

In [187]:
# predict_file(path + 'mytest/test/4.jpg')

In [188]:
test_dir = path + 'mytest/test/'
with open(path + 'mytest/result.csv', 'w') as file:
    count = 0
    for f in os.listdir(test_dir):
        count++
        if count % 100 == 0:
            print("count: %d" % count)
        id = f.split(".")[0]
        test_file = test_dir + f
        c = predict_file(test_file)
#         print('%s %s' % (id, c))
        file.write('%s,%s\n' % (id, c))

2 0
8 1
9 1
1 0
4 0
20 1
